## Calculadora de Investimentos do Aprendiz Moderno

Utilizando dados históricos para calcular a performance de uma carteira fictícia

In [14]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt
pd.set_option('display.max_rows', 1000)

In [15]:
def dividendo_por_acao(lista, data):
    dividendo = lista[(lista.index.month == data.month) & (lista.index.year == data.year)]
    dividendo = dividendo.sum()
    return dividendo


def calcula_investimento(ticker, inicio='2010-01-01', termino='2020-01-01', aporte_mensal=1000.0):
    lista_de_dividendos = yf.Ticker(ticker).dividends
    hist = yf.download(ticker, start=inicio, end=termino, interval='1mo')
    hist = hist.dropna()
    hist['valor_investido'] = 0.00
    hist['aporte_mensal'] = aporte_mensal
    hist['aporte_total'] = 0.00
    hist['total_acoes'] = 0.00
    hist['resto'] = 0.00
    hist['dividendos'] = 0.00
    total_de_linhas = hist.shape[0]

    for i in range(0, total_de_linhas):
        linha = hist.iloc[i,:]
        data = hist.index[i]
        valor_de_fechamento = linha['Adj Close']
        if i > 0:
            linha_anterior = hist.iloc[i-1, :]
            acoes_anterior = linha_anterior['total_acoes']
            resto_anterior = linha_anterior['resto']
        else:
            # i == 0
            acoes_anterior = 0.00
            resto_anterior = 0.00
            
        dividendo = dividendo_por_acao(lista_de_dividendos, data)
        total_dividendos = dividendo*acoes_anterior
        aporte_total = aporte_mensal + resto_anterior + total_dividendos
        novas_acoes = int(aporte_total/valor_de_fechamento)
        total_acoes = acoes_anterior + novas_acoes
        valor_investido = total_acoes*valor_de_fechamento
        resto = aporte_total - novas_acoes*valor_de_fechamento
        # substituir valores
        hist.at[data, 'valor_investido'] = valor_investido
        hist.at[data, 'aporte_total'] = aporte_total
        hist.at[data, 'total_acoes'] = total_acoes
        hist.at[data, 'resto'] = resto
        hist.at[data, 'dividendos'] = total_dividendos

    return hist

In [25]:
carteira = [
    {
        "ticker":"ITUB3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":200.0
    },
    {
        "ticker":"ABEV3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"B3SA3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"WEGE3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"EGIE3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":250.0
    }
]

In [39]:
def calcula_carteira(portfolio):
    COLS = ['data', 'ticker', 'valor_investido', 'aporte_mensal', 'aporte_total', 'total_acoes', 'dividendos']
    resultado = pd.DataFrame(columns=COLS)
    if len(portfolio) > 0:
        for acao in portfolio:
            try:
                hist = calcula_investimento(
                    ticker=acao['ticker'], inicio=acao['inicio'], termino=acao['termino'], aporte_mensal=acao['aporte_mensal']
                    )
                print('Ativo calculado!')
                hist = hist.reset_index()
                hist = hist.rename(columns={'Date':'data'})
                hist['ticker'] = acao['ticker']
                resultado = resultado.append(hist[COLS], ignore_index=True)
            except:
                print('Não foi possível calcular o ticker: ', acao['ticker'])
                pass
        return resultado
    else:
        print('A carteira precisa ter pelo menos 1 ativo.')

In [40]:
calcula_carteira(carteira)

[*********************100%***********************]  1 of 1 completed
Ativo calculado!
[*********************100%***********************]  1 of 1 completed
Ativo calculado!
[*********************100%***********************]  1 of 1 completed
Ativo calculado!
[*********************100%***********************]  1 of 1 completed
Ativo calculado!
[*********************100%***********************]  1 of 1 completed
Ativo calculado!


,data,ticker,valor_investido,aporte_mensal,aporte_total,total_acoes,dividendos
0,2010-01-01,ITUB3.SA,190.865707,200.0,200.000000,15.0,0.000000
1,2010-02-01,ITUB3.SA,397.898689,200.0,209.314293,31.0,0.180000
2,2010-03-01,ITUB3.SA,617.138474,200.0,204.319227,46.0,0.372000
3,2010-04-01,ITUB3.SA,806.100117,200.0,203.630421,61.0,0.552000
4,2010-05-01,ITUB3.SA,942.641094,200.0,206.141080,77.0,0.732000
5,2010-06-01,ITUB3.SA,1092.470012,200.0,211.191606,95.0,0.924000
6,2010-07-01,ITUB3.SA,1485.555153,200.0,205.337288,110.0,1.140000
7,2010-08-01,ITUB3.SA,1596.065760,200.0,204.081586,125.0,1.320000
8,2010-09-01,ITUB3.SA,1360.601662,200.0,214.053694,148.0,1.500000
9,2010-10-01,ITUB3.SA,1600.425049,200.0,204.384842,169.0,1.776000
